<a href="https://colab.research.google.com/github/murni27/Bank-loan-repayment-prediction/blob/main/home_credit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"angelinamurni87","key":"f3a111b04196c8c3a80427d3b6e0ea54"}'}

In [ ]:
# Let's make sure the kaggle.json file is present.
!ls -lha kaggle.json

-rw-r--r-- 1 root root 71 Apr 20 07:16 kaggle.json


In [ ]:
# Next, install the Kaggle API client.
!pip install -q kaggle

In [ ]:
# The Kaggle API client expects this file to be in ~/.kaggle,
# so move it there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# List available datasets.
!kaggle datasets list

ref                                                            title                                             size  lastUpdated          downloadCount  voteCount  usabilityRating  
-------------------------------------------------------------  -----------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
datasets/muratkokludataset/date-fruit-datasets                 Date Fruit Datasets                              408KB  2022-04-03 09:25:39           1412        269  0.9375           
datasets/piterfm/2022-ukraine-russian-war                      2022 Ukraine Russia War                            2KB  2022-04-19 09:32:41           9639        526  1.0              
datasets/muratkokludataset/acoustic-extinguisher-fire-dataset  Acoustic Extinguisher Fire Dataset               621KB  2022-04-02 22:59:36            144        236  0.9375           
datasets/kamilpytlak/personal-key-indicators-of-heart-disease  Personal Key Indi

In [ ]:
!kaggle competitions download -c home-credit-default-risk

 97% 668M/688M [00:04<00:00, 111MB/s]
100% 688M/688M [00:04<00:00, 156MB/s]


In [ ]:
!unzip \*.zip

Archive:  home-credit-default-risk.zip
  inflating: HomeCredit_columns_description.csv  
  inflating: POS_CASH_balance.csv    
  inflating: application_test.csv    
  inflating: application_train.csv   
  inflating: bureau.csv              
  inflating: bureau_balance.csv      
  inflating: credit_card_balance.csv  
  inflating: installments_payments.csv  
  inflating: previous_application.csv  
  inflating: sample_submission.csv   


In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd

In [ ]:
df = pd.read_csv('application_train.csv')

In [ ]:
training_data, testing_data = train_test_split(df, test_size=0.2, random_state=25)

In [ ]:
training_data.describe()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
count,246008.000000,246008.000000,246008.000000,2.460080e+05,2.460080e+05,246000.000000,2.457800e+05,246008.000000,246008.000000,246008.00000,...,246008.000000,246008.000000,246008.000000,246008.000000,212850.000000,212850.000000,212850.000000,212850.000000,212850.000000,212850.000000
mean,278274.234322,0.081091,0.416275,1.688343e+05,5.989584e+05,27104.847329,5.384404e+05,0.020830,-16036.089623,63737.39189,...,0.008130,0.000589,0.000533,0.000313,0.006540,0.007066,0.034550,0.267089,0.265868,1.896848
std,102790.196964,0.272975,0.720659,2.587828e+05,4.024649e+05,14504.140292,3.695062e+05,0.013797,4362.463669,141210.97297,...,0.089798,0.024271,0.023070,0.017689,0.084863,0.113280,0.206631,0.912912,0.834023,1.867626
min,100002.000000,0.000000,0.000000,2.565000e+04,4.500000e+04,1615.500000,4.050000e+04,0.000290,-25229.000000,-17583.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,189280.500000,0.000000,0.000000,1.125000e+05,2.700000e+05,16506.000000,2.385000e+05,0.010006,-19675.000000,-2759.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,278379.500000,0.000000,0.000000,1.482368e+05,5.135310e+05,24903.000000,4.500000e+05,0.018850,-15755.000000,-1216.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,367190.250000,0.000000,1.000000,2.025000e+05,8.086500e+05,34596.000000,6.795000e+05,0.028663,-12412.000000,-291.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
max,456255.000000,1.000000,19.000000,1.170000e+08,4.050000e+06,258025.500000,4.050000e+06,0.072508,-7676.000000,365243.00000,...,1.000000,1.000000,1.000000,1.000000,4.000000,9.000000,8.000000,27.000000,261.000000,25.000000


In [ ]:
testing_data.describe()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
count,61503.000000,61503.000000,61503.000000,6.150300e+04,6.150300e+04,61499.000000,6.145300e+04,61503.000000,61503.000000,61503.000000,...,61503.000000,61503.000000,61503.000000,61503.000000,53142.000000,53142.000000,53142.000000,53142.000000,53142.000000,53142.000000
mean,277805.661691,0.079281,0.420158,1.686523e+05,5.992964e+05,27123.480471,5.382194e+05,0.021021,-16040.616783,64125.656911,...,0.008130,0.000618,0.000406,0.000423,0.005852,0.006737,0.033608,0.268620,0.263897,1.912499
std,102790.070139,0.270179,0.727940,1.151685e+05,4.025974e+05,14452.157905,3.692103e+05,0.013967,4370.116689,141535.363436,...,0.089798,0.024849,0.020158,0.020557,0.079656,0.100017,0.196695,0.928287,0.608198,1.875930
min,100012.000000,0.000000,0.000000,2.610000e+04,4.500000e+04,1980.000000,4.500000e+04,0.000533,-25196.000000,-17912.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,188515.500000,0.000000,0.000000,1.125000e+05,2.700000e+05,16580.250000,2.385000e+05,0.010006,-19701.000000,-2763.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,277449.000000,0.000000,0.000000,1.440000e+05,5.146020e+05,24934.500000,4.500000e+05,0.018850,-15735.000000,-1203.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,366975.500000,0.000000,1.000000,2.025000e+05,8.086500e+05,34596.000000,6.795000e+05,0.028663,-12416.000000,-283.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
max,456252.000000,1.000000,14.000000,1.350000e+07,4.050000e+06,225000.000000,4.050000e+06,0.072508,-7489.000000,365243.000000,...,1.000000,1.000000,1.000000,1.000000,3.000000,6.000000,8.000000,23.000000,6.000000,17.000000
